In [ ]:
import random
import shap
from lime import lime_image
from skimage.segmentation import mark_boundaries
import cv2

# ------------------------------
# 7a. Feature Maps (per Conv2D layer)
# ------------------------------
# Pick a random image from validation set
batch_idx = random.randint(0, len(val_ds)-1)
img, label = val_ds[batch_idx][0][0], val_ds[batch_idx][1][0]  # first image in batch
img_input = np.expand_dims(img, axis=0)

# Detect all Conv2D layers automatically
conv_layers = [layer for layer in model.layers if 'conv' in layer.name]

for layer in conv_layers:
    activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer.output)
    activations = activation_model.predict(img_input)

    # Plot first 6 filters
    plt.figure(figsize=(15, 5))
    for i in range(min(6, activations.shape[-1])):
        plt.subplot(1, 6, i+1)
        plt.imshow(activations[0, :, :, i], cmap='viridis')
        plt.axis('off')
    plt.suptitle(f"Feature Maps of {layer.name}")
    plt.show()

# ------------------------------
# 7b. SHAP (local interpretability)
# ------------------------------
sample_imgs, sample_labels = val_ds[0][:5]  # first 5 images from first batch
explainer = shap.GradientExplainer(model, sample_imgs)
shap_values = explainer.shap_values(sample_imgs)
shap.image_plot(shap_values, sample_imgs)

# Optional: LIME explanation for one image
explainer_lime = lime_image.LimeImageExplainer()
img_to_explain = val_ds[0][0][0]
explanation = explainer_lime.explain_instance(
    np.array(img_to_explain),
    classifier_fn=lambda x: model.predict(x),
    top_labels=1,
    hide_color=0,
    num_samples=1000
)
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False
)
plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.title("LIME Explanation")
plt.axis('off')
plt.show()

# ------------------------------
# 7c. Grad-CAM
# ------------------------------
# Automatically get the last Conv2D layer
last_conv_layer = conv_layers[-1].name

def get_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_outputs)
    conv_outputs = conv_outputs[0]
    weights = tf.reduce_mean(grads[0], axis=(0, 1))

    cam = np.zeros(conv_outputs.shape[:2], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * conv_outputs[:, :, i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam.numpy(), (img_width, img_height))
    cam = cam - cam.min()
    cam = cam / cam.max()
    return cam

cam = get_gradcam(model, np.expand_dims(img, axis=0), last_conv_layer)

# Overlay heatmap on original image
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
superimposed_img = heatmap * 0.4 + img
plt.imshow(superimposed_img / 255)
plt.title("Grad-CAM Overlay")
plt.axis('off')
plt.show()
